# Dependencies

In [ ]:
import numpy as np
import pandas as pd
import os

import cv2
from google.colab.patches import cv2_imshow

import PIL
from PIL import Image

import dlib
from skimage import io

from numpy import asarray

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

# Data

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Deep_Fake/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Deep_Fake/test.csv')
df_test_all = pd.read_csv('/content/drive/MyDrive/Deep_Fake/test_all.csv')

# train_img_df = pd.read_csv('/content/drive/MyDrive/Deep_Fake/train_img.csv')
# test_img_df = pd.read_csv('/content/drive/MyDrive/Deep_Fake/test_img.csv')

with open('/content/drive/MyDrive/Deep_Fake/data_all.pkl', 'rb') as f:
    dic_all = pickle.load(f)

# Img exteration

In [ ]:
imgPath=[]
resize= 1
frame_count_real = []
frame_count_fake = []


# Function to extract frames
def FrameCapture(vid_path , vid_name , n_frames):
    imgPath = []
    v_cap = cv2.VideoCapture(vid_path)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Pick 'n_frames' evenly spaced frames to sample
    sample = np.linspace(0, v_len - 1, n_frames).astype(int) #Index in sample array are the frames which will be extracted.

    # Loop through frames
    
    frames = []
    for j in range(v_len):
        success = v_cap.grab()
        if j in sample:
            # Load frame
            success, frame = v_cap.read()
            if not success:          
                continue
            
            frame = Image.fromarray(frame)
            
            # Resize frame to desired size
            if resize is not None:
                frame = frame.resize([int(d * resize) for d in frame.size])
                frame = np.asarray(frame)
            frames.append(frame)
    
    # Used as counter variable
  
    # checks whether frames were extracted
    currentCount = 0
    success = 1
  
    while success and currentCount<min(n_frames,len(frames)):
        cv2.imwrite("/content/drive/MyDrive/Deep_Fake/Frames/Train/" + vid_name + '_' + str(currentCount) + '.png' ,frames[int(currentCount)])
        imgPath.append("/content/drive/MyDrive/Deep_Fake/Frames/Train/" + vid_name + '_' + str(currentCount))
        currentCount += 1

    return imgPath

In [ ]:
n_frames = 11
resize= 1

def captureFrame(df):
  imgPath = []
  video_label = []
  vid_name = []

  for row in df.iterrows():
      video_name = row[1][1]
      video_path = row[1][-2]
      category = row[1][-1]

      img_path = FrameCapture(video_path , video_name , n_frames)
      label = [category] * len(img_path)
      vd_name = [video_name] * len(img_path)

      imgPath = imgPath + img_path
      video_label = video_label + label
      vid_name = vid_name + vd_name

  return imgPath , video_label

In [ ]:
train_img_path, train_img_label = captureFrame(df_train)

In [ ]:
train_img_path = [s + '.png' for s in train_img_path]
test_img_path = [s + '.png' for s in test_img_path]

In [ ]:
dic_train = {'train_img_path': train_img_path , 'train_img_label':train_img_label}
train_img_df = pd.DataFrame(dic_train)
train_img_df.to_csv('/content/drive/MyDrive/Deep_Fake/train_img.csv')

In [ ]:
test_img_path, test_img_label = captureFrame(df_test)

In [ ]:
dic_test = {'test_img_path': test_img_path , 'test_img_label':test_img_label}
test_img_df = pd.DataFrame(dic_test)
test_img_df.to_csv('/content/drive/MyDrive/Deep_Fake/test_img.csv')

# Face Detection

In [ ]:
def detect_faces(image):
    # Create a face detector
    face_detector = dlib.get_frontal_face_detector()

    # Run detector and get bounding boxes of the faces on image.
    detected_faces = face_detector(image, 1)
    face_frames = [(x.left(), x.top(), x.right(), x.bottom()) for x in detected_faces]

    return face_frames


def faceDetect_DLIB(imgPath, imgLabel):
  processedImagePath_dlib = []
  processedImageLab_dlib = []

  for i in range(len(imgPath)):
      # Load image
      img_path = imgPath[i]
      image = io.imread(img_path)

      label = imgLabel[i]

      # Detect faces
      detected_faces = detect_faces(image)

      # Crop faces and plot
      for n, face_rect in enumerate(detected_faces):
          face = Image.fromarray(image).crop(face_rect)
          final_face = asarray(face)
          resize_face = cv2.resize(final_face,(64, 64), interpolation=cv2.INTER_CUBIC)
          finalImg = cv2.cvtColor(resize_face, cv2.COLOR_BGR2RGB)
          #cv2.imwrite("/content/drive/MyDrive/Deep_Fake/Frames/FD_test_/" + str(i+1) + '.png' , finalImg)
          processedImagePath_dlib.append("/content/drive/MyDrive/Deep_Fake/Frames/FD_train/" + str(i+1) + '.png' )
          processedImageLab_dlib.append(label)

  return processedImagePath_dlib , processedImageLab_dlib

In [ ]:
fd_train_img_path , fd_train_img_label = faceDetect_DLIB(dic_all['train_img_path'] , dic_all['train_img_label'])
dic_all['fd_train_img_label'] = fd_train_img_label

In [ ]:
fd_test_img_path , fd_test_img_label = faceDetect_DLIB(dic_all['test_img_path'] , dic_all['test_img_label'])
dic_all['fd_test_img_label'] = fd_test_img_label

In [ ]:
with open('/content/drive/MyDrive/Deep_Fake/data_all.pkl', 'wb') as f:
    pickle.dump(dic_all, f)

# Data Augmentation

In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from skimage.util import random_noise
import numpy as np
import torch
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import argparse

In [ ]:
def noisy(noise_typ,image):
  if noise_typ == "gauss":
    mean = 0
    var = 100
    sigma = var ** 0.5
    row, col, _ = image.shape
    gaussian = np.random.normal(mean, sigma, (row, col)) 

    noisy = np.zeros(image.shape, np.float32)

    if len(image.shape) == 2:
        noisy = image + gaussian
    else:
        noisy[:, :, 0] = image[:, :, 0] + gaussian
        noisy[:, :, 1] = image[:, :, 1] + gaussian
        noisy[:, :, 2] = image[:, :, 2] + gaussian

    cv2.normalize(noisy, noisy, 0, 255, cv2.NORM_MINMAX, dtype=-1)
    noisy = noisy.astype(np.uint8)
    return noisy
  elif noise_typ == "s&p":
    row,col,ch = image.shape
    s_vs_p = 0.5
    amount = 0.05
    out = np.copy(image)
    # Salt mode
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    out[coords] = 1

    # Pepper mode
    num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    out[coords] = 0
    return out

  elif noise_typ =="speckle":
    row,col,ch = image.shape
    gauss = np.random.randn(row,col,ch)
    gauss = gauss.reshape(row,col,ch)
    noisy = image + image * gauss
    return noisy

def create_noisy_img(face_detector_arr):
    arr=[]
    for i in range(len(face_detector_arr)):
        img=cv2.imread(face_detector_arr[i])
        
        noisy_img = noisy("gauss", img)
        noisy_img = cv2.resize(noisy_img, (64, 64))
        
        cv2.imwrite("/content/drive/MyDrive/Deep_Fake/Frames/noise/Test/" + str(i+1) + '.png' ,noisy_img)
        arr.append("/content/drive/MyDrive/Deep_Fake/Frames/noise/Test/" + str(i+1) + '.png' )
    return arr

In [ ]:
dic_all.keys()

dict_keys(['train_img_path', 'train_img_label', 'test_img_path', 'test_img_label', 'fd_train_img_path', 'fd_test_img_path'])

In [ ]:
fd_train_img_path_noise = create_noisy_img(dic_all['fd_train_img_path'])

In [ ]:
dic_all['fd_train_img_path_noise'] = fd_train_img_path_noise

In [ ]:
fd_test_img_path_noise = create_noisy_img(dic_all['fd_test_img_path'])

In [ ]:
dic_all['fd_test_img_path_noise'] = fd_test_img_path_noise

In [ ]:
with open('/content/drive/MyDrive/Deep_Fake/data_all.pkl', 'wb') as f:
    pickle.dump(dic_all, f)

# Train CNN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import keras


In [ ]:
def img_to_tensor(img_path):
    img = Image.open(img_path)
    img_resize = img.convert('RGB').resize((128, 128))
    tensor = np.round(np.array(img_resize,dtype=np.float32))
    data_tensor = tf.convert_to_tensor(tensor)
    return data_tensor

In [ ]:
def create_data(arr):
    re_arr = []
    for i in arr:
        re_arr.append(img_to_tensor(i))
    return re_arr

dic_all['X_train_tensor'] = create_data(dic_all['fd_train_img_path_noise'])
dic_all['X_test_tensor'] = create_data(dic_all['fd_test_img_path_noise'])

In [ ]:
def create_label(arr):
    x = []
    for i in range(len(arr)):
        if(arr[i]==1):
            x.append([0,1])
        else:
            x.append([1,0])

    return x

dic_all['Y_train'] = create_label(dic_all['fd_train_img_label'])
dic_all['Y_test'] = create_label(dic_all['fd_test_img_label'])

In [ ]:
with open('/content/drive/MyDrive/Deep_Fake/data_all.pkl', 'wb') as f:
    pickle.dump(dic_all, f)

In [ ]:
# Parameters
BATCH_SIZE = 50
max_epochs = 5

# Datasets

X_train = dic_all['X_train_tensor']
X_test = dic_all['X_test_tensor']
Y_train = dic_all['Y_train']
Y_test = dic_all['Y_test']

# Generators
training_generator = tf.data.Dataset.from_tensor_slices((X_train , Y_train))
test_generator = tf.data.Dataset.from_tensor_slices((X_test , Y_test))

#Augment and shuffle
def augment(points, label):
    # jitter points
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float32)
    # shuffle points
    points = tf.random.shuffle(points)
    return points, label

# train_dataset = training_generator.shuffle(len(training_set[0])).map(augment).batch(BATCH_SIZE)
# test_dataset = test_generator.shuffle(len(test_set[0])).batch(BATCH_SIZE)
# val_dataset = validation_generator.shuffle(len(validation_set[0])).batch(BATCH_SIZE)

train_dataset = training_generator.batch(BATCH_SIZE)
test_dataset = test_generator.batch(BATCH_SIZE)

In [ ]:
train_dataset

<BatchDataset shapes: ((None, 128, 128, 3), (None, 2)), types: (tf.float32, tf.int32)>

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
def create_model():
    model= Sequential()
    model.add(Conv2D(filters=16, kernel_size=2, activation='tanh', input_shape=(128,128,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=32,kernel_size = 2,activation='tanh'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    # model.add(Conv2D(filters=64,kernel_size = 2,activation='tanh'))
    # model.add(MaxPooling2D(pool_size=2))
    # model.add(Conv2D(filters=128,kernel_size = 2,activation='tanh'))
    # model.add(MaxPooling2D(pool_size=2))

    model.add(Flatten())

    # model.add(Dense(50,activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(2,activation = 'softmax'))
        
    model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
    return model

In [ ]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 127, 127, 16)      208       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 62, 32)        2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 31, 31, 32)        0

In [ ]:
model.fit(train_dataset , epochs=3)

Epoch 1/3
95/95 [==============================] - 13s 31ms/step - loss: 0.7402 - accuracy: 0.6997
Epoch 2/3
95/95 [==============================] - 3s 29ms/step - loss: 0.6753 - accuracy: 0.7055
Epoch 3/3
95/95 [==============================] - 3s 28ms/step - loss: 0.6640 - accuracy: 0.7055


In [ ]:
model.evaluate(test_dataset)

10/10 [==============================] - 1s 25ms/step - loss: 0.6531 - accuracy: 0.7271


[0.6531263589859009, 0.7270875573158264]

In [ ]:
checkpoint_path = "/content/drive/MyDrive/Deep_Fake/model_cnn.ckpt"

In [ ]:
#model.save_weights(checkpoint_path)

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
model.evaluate(test_dataset)

10/10 [==============================] - 9s 24ms/step - loss: 0.6531 - accuracy: 0.7271


[0.6531263589859009, 0.7270875573158264]

# Prediction

In [ ]:
j = 0
arr_ = []
for i in test_dataset:
    arr = i[0][j]
    arr_.append(arr)
    j+=1
    if j > 10:
        break

In [ ]:
arr_ = tf.expand_dims(arr, axis=0)
print(arr_.shape)

(1, 128, 128, 3)


In [ ]:
arr__ = tf.stack(arr_)
print(arr__.shape)

(10, 128, 128, 3)


In [ ]:
p = model.predict(arr__)

In [ ]:
p

array([[0.5920801 , 0.40791988],
       [0.5920801 , 0.40791988],
       [0.5920801 , 0.40791988],
       [0.5920801 , 0.40791988],
       [0.5920801 , 0.40791988],
       [0.5920801 , 0.40791988],
       [0.5920801 , 0.40791988],
       [0.5920801 , 0.40791988],
       [0.5920801 , 0.40791988],
       [0.5920801 , 0.40791988]], dtype=float32)

In [ ]:
final_ans(arr__ , model)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
The video is FAKE!!


# Analysis

In [ ]:
a = dic_all.keys()
for i in a:
    print(i , len(dic_all[i]))

train_img_path 4937
train_img_label 4937
test_img_path 536
test_img_label 536
fd_train_img_path 4716
fd_test_img_path 491
fd_train_img_path_noise 4716
fd_test_img_path_noise 491
fd_test_img_label 491
fd_train_img_label 4716
X_train 4716
X_test 491
X_train_tensor 4716
X_test_tensor 491


# Pipeline


In [ ]:
import numpy as np
import pandas as pd
from numpy import asarray

import cv2 
import PIL
from PIL import Image

import dlib
from skimage import io

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
def df_detect_2(vid_path , model):
    imgPath = []
    n_frames = 11
    resize= 1
    v_cap = cv2.VideoCapture(vid_path)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    sample = np.linspace(0, v_len - 1, n_frames).astype(int) 

    frames = []
    for j in range(v_len):
        success = v_cap.grab()
        if j in sample:
            success, frame = v_cap.read()
            if not success:          
                continue
            
            frame = Image.fromarray(frame)
            
            if resize is not None:
                frame = frame.resize([int(d * resize) for d in frame.size])
                frame = np.asarray(frame)
            frames.append(frame)
    
    currentCount = 0
    success = 1
  
    while success and currentCount<min(n_frames,len(frames)):
        cv2.imwrite("/content/img_" + str(currentCount) + '.png' ,frames[int(currentCount)])
        imgPath.append("/content/img_" + str(currentCount) + '.png')
        currentCount += 1

    face_tensor = []

    for i in range(len(imgPath)):
        image = io.imread(imgPath[i])

        face_detector = dlib.get_frontal_face_detector()
        detected_faces = face_detector(image, 1)
        face_frames = [(x.left(), x.top(), x.right(), x.bottom()) for x in detected_faces]
        
        for n, face_rect in enumerate(face_frames):
            face = Image.fromarray(image).crop(face_rect)
            final_face = asarray(face)
            resize_face = cv2.resize(final_face,(128, 128), interpolation=cv2.INTER_CUBIC)
            finalImg = cv2.cvtColor(resize_face, cv2.COLOR_BGR2RGB)

            tensor = np.round(np.array(finalImg,dtype=np.float32))
            data_tensor = tf.convert_to_tensor(tensor)
            data_tensor_ = tf.expand_dims(data_tensor, axis=0)
            face_tensor.append(data_tensor_)

    count = 0
    for i in range(len(face_tensor)):
        p =  model.predict(face_tensor[i])
        pr = 0 if p[0][0] > p[0][1] else 1
        count += pr
    if count < 6:
        print('The video is FAKE!!')
    else :
        print('The video is REAL!!')

In [ ]:
def create_model():
    model= Sequential()
    model.add(Conv2D(filters=16, kernel_size=2, activation='tanh', input_shape=(128,128,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=32,kernel_size = 2,activation='tanh'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))

    model.add(Flatten())

    model.add(Dense(10,activation='relu'))
    model.add(Dense(2,activation = 'softmax'))
        
    model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
    return model

In [ ]:
model = create_model()
checkpoint_path = "/content/drive/MyDrive/Deep_Fake/model_cnn.ckpt"
model.load_weights(checkpoint_path)

In [ ]:
import pickle
pickle.dump(model,open("/content/drive/MyDrive/Deep_Fake/new_model.pkl",'wb'))

INFO:tensorflow:Assets written to: ram://a0c24bf2-831b-46f4-be05-fe2e4ffcf93b/assets


In [ ]:
model_json = model.to_json()
with open('/content/model.json' , 'w') as json_file:
    json_file.write(model_json)
model.save_weights('/content/model.h5')
model.save('/content/modelf.h5')

In [ ]:
path = df_test.loc[0][3]

In [ ]:

model_f = pickle.load(open('/content/drive/MyDrive/Deep_Fake/new_model.pkl','rb'))

In [ ]:
df_detect_2(path , model)

The video is REAL!!


In [ ]:
df_detect_2(path , model_f)

The video is REAL!!
